In [119]:
%load_ext autoreload
%autoreload 2
from parse.parse_reactions import ReactionParser
from parse.parse_species import parse_species_string, parse_all_species
from parse.parse_rate_constant import RateConstantParser
from parse.parse_input import separate_input_file, parse_args_kwargs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
sections = separate_input_file('input/sample_input.txt')
species_section = sections['species']
all_species, equivalent_species = parse_all_species(species_section)
print(all_species)

{'general': [], 'species': [], 'rate constants': [], 'reactions': []}
[Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005), Species(name='R', type=<SpeciesType.UNIT: 1>, FW=25.0, c0=0.0), Species(name='A', type=<SpeciesType.UNIT: 1>, FW=100.0, c0=1.5), Species(name='B', type=<SpeciesType.UNIT: 1>, FW=200.0, c0=1.5), Species(name='P[R.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[R.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[A.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[A.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[B.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='P[B.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), Species(name='D', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0)]


In [121]:
species_string = 'U I      c0=0.005 FW=50.0'
parse_species_string(species_string)

Species(name='I', type=<SpeciesType.UNIT: 1>, FW=50.0, c0=0.005)

In [122]:
constants_section = sections['rate constants']
rcp = RateConstantParser()
rcp.parse_all_constants(constants_section)

In [123]:
reaction_string = 'PR 23P[A.A] + B -kpAAB-> 5P[A.B]'
rp = ReactionParser(all_species, rcp)
rp.parse_reaction_string(reaction_string)

('PR',
 [ReactionSpecies(species=Species(name='P[A.A]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), coeff=23),
  ReactionSpecies(species=Species(name='B', type=<SpeciesType.UNIT: 1>, FW=200.0, c0=1.5), coeff=1)],
 [ReactionSpecies(species=Species(name='P[A.B]', type=<SpeciesType.POLY: 2>, FW=1.0, c0=0.0), coeff=5)],
 ['kpAAB'], {'k': 1000.0})

In [124]:
reactions_section = sections['reactions']
all_reactions = rp.parse_all_reactions(reactions_section)
print(all_reactions)

[<reactions.InitiatorDecompositionReaction object at 0x1085dfb10>, <reactions.InitiationReaction object at 0x108704e50>, <reactions.InitiationReaction object at 0x1080d1e10>, <reactions.PropagationReaction object at 0x1085c6a90>, <reactions.PropagationReaction object at 0x108579190>, <reactions.PropagationReaction object at 0x108773950>, <reactions.PropagationReaction object at 0x108607f10>, <reactions.PropagationReaction object at 0x108594750>, <reactions.PropagationReaction object at 0x108594450>, <reactions.PropagationReaction object at 0x1085972d0>, <reactions.PropagationReaction object at 0x1085b4dd0>, <reactions.PropagationReaction object at 0x108679010>, <reactions.PropagationReaction object at 0x108678250>, <reactions.PropagationReaction object at 0x108679e10>, <reactions.PropagationReaction object at 0x108679510>, <reactions.DepropagationReaction object at 0x10867a4d0>, <reactions.TerminationCombinationReaction object at 0x1085a9150>, <reactions.TerminationCombinationReaction 

In [125]:
str = "ktdBB A=1e5 E=1e-3"
r = rcp.parse_constants_string(str)
r


['ktdBB'], {'A': 100000.0, 'E': 0.001}

In [126]:
c = rcp.parse_reaction_constant("PR P[R.A] + A -kpAAA-> P[A.A]")
print(c)

['kpAAA'], {'k': 1000.0}


In [127]:
from itertools import product

# Given string format
template_string = 'P[-.A] + P[-.B] -kpABC-> D'

# Define possible replacements for "P[-.A]" and "P[-.B]"
replacements_A = ['P[A.A]', 'P[B.A]']
replacements_B = ['P[A.B]', 'P[B.B]']

# Generate all permutations
permutations = list(product(replacements_A, replacements_B))

# Create a list of strings with permutations
for p in permutations:
    print(p)
result_strings = [template_string.replace('P[-.A]', p[0]).replace('P[-.B]', p[1]) for p in permutations]

# Print the result
for idx, result_string in enumerate(result_strings, start=1):
    print(f'{idx}: {result_string}')


('P[A.A]', 'P[A.B]')
('P[A.A]', 'P[B.B]')
('P[B.A]', 'P[A.B]')
('P[B.A]', 'P[B.B]')
1: P[A.A] + P[A.B] -kpABC-> D
2: P[A.A] + P[B.B] -kpABC-> D
3: P[B.A] + P[A.B] -kpABC-> D
4: P[B.A] + P[B.B] -kpABC-> D
